# Google Drive Transfer Tool with rclone

This notebook provides a simple graphical interface for transferring files between Google Drive remotes using rclone. Perfect for migrating files between different Google accounts or organizing your data.

## 📋 Prerequisites
- You need an `rclone.conf` file with your Google Drive configurations
- If you don't have one, see the [rclone documentation](https://rclone.org/drive/) for setup instructions

## 🚀 Getting Started
1. Run the setup cell below
2. Upload your rclone.conf file
3. Configure your transfer settings
4. Start the transfer

---

## Setup: Install Dependencies

Run this cell first to install rclone and required Python libraries.

In [ ]:
# Install rclone and dependencies
import subprocess
import sys
import os
from pathlib import Path

print("🔧 Installing dependencies...")

# Install rclone
try:
    result = subprocess.run(['which', 'rclone'], capture_output=True, text=True)
    if result.returncode != 0:
        print("📦 Installing rclone...")
        # Use subprocess instead of shell magic
        subprocess.run(['bash', '-c', 'curl https://rclone.org/install.sh | sudo bash'], check=True)
    else:
        print("✅ rclone already installed")
except Exception as e:
    print(f"⚠️  Installing rclone manually: {e}")
    # Fallback to manual installation
    subprocess.run(['bash', '-c', 'curl https://rclone.org/install.sh | sudo bash'], check=False)

# Install Python packages
print("📦 Installing Python packages...")
subprocess.run([sys.executable, '-m', 'pip', 'install', '-q', 'ipywidgets'], check=False)

# Enable ipywidgets for Colab
try:
    from google.colab import output
    output.enable_custom_widget_manager()
    print("✅ ipywidgets enabled for Colab")
except ImportError:
    print("ℹ️  Not running in Colab, widgets should work normally")

# Create rclone config directory
config_dir = Path('/root/.config/rclone')
config_dir.mkdir(parents=True, exist_ok=True)

print("\n✅ Setup complete! You can now proceed to the next steps.")

## Step 1: Configure rclone

Choose to either upload an existing rclone.conf file or create a new configuration with Google Drive remotes.

In [ ]:
import ipywidgets as widgets
from IPython.display import display, clear_output, HTML
import io
import os
import subprocess
import re
import json
import urllib.parse
import requests
from pathlib import Path

# Global variables
config_ready = False
current_remotes = []

# Main container
main_container = widgets.VBox()

# Configuration method selection
config_method = widgets.RadioButtons(
    options=[
        ('📤 Upload existing rclone.conf file', 'upload'),
        ('🆕 Create new rclone configuration', 'create')
    ],
    value='upload',
    description='Configuration Method:',
    style={'description_width': 'initial'},
    layout=widgets.Layout(margin='10px 0')
)

# Upload section widgets
upload_widget = widgets.FileUpload(
    accept='.conf',
    multiple=False,
    description='Choose rclone.conf',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='300px')
)

# Create section widgets
remote_name_input = widgets.Text(
    placeholder='e.g., my_drive, backup_drive',
    description='Remote Name:',
    style={'description_width': '120px'},
    layout=widgets.Layout(width='400px')
)

auth_method = widgets.RadioButtons(
    options=[
        ('🔐 Use built-in OAuth (Recommended)', 'builtin'),
        ('🔑 Use custom client ID/secret', 'custom')
    ],
    value='builtin',
    description='Authentication:',
    style={'description_width': 'initial'}
)

client_id_input = widgets.Text(
    placeholder='Your Google API Client ID',
    description='Client ID:',
    style={'description_width': '120px'},
    layout=widgets.Layout(width='500px')
)

client_secret_input = widgets.Password(
    placeholder='Your Google API Client Secret',
    description='Client Secret:',
    style={'description_width': '120px'},
    layout=widgets.Layout(width='500px')
)

add_remote_button = widgets.Button(
    description='➕ Add Remote',
    button_style='primary',
    layout=widgets.Layout(width='150px')
)

# Status and output areas
status_output = widgets.Output()
remotes_display = widgets.Output()

def show_upload_section():
    """Display the upload configuration section"""
    upload_section = widgets.VBox([
        HTML("<div style='margin: 15px 0; padding: 10px; background-color: #f0f8ff; border-left: 4px solid #2196F3;'>"
             "<b>📤 Upload Method</b><br/>Upload your existing rclone.conf file that contains your Google Drive configurations."
             "</div>"),
        upload_widget
    ])
    return upload_section

def show_create_section():
    """Display the create configuration section"""
    custom_auth_widgets = widgets.VBox([
        client_id_input,
        client_secret_input
    ])
    
    def toggle_custom_auth(change):
        custom_auth_widgets.layout.display = 'block' if change['new'] == 'custom' else 'none'
    
    auth_method.observe(toggle_custom_auth, names='value')
    custom_auth_widgets.layout.display = 'none'  # Initially hidden
    
    create_section = widgets.VBox([
        HTML("<div style='margin: 15px 0; padding: 10px; background-color: #f0fff0; border-left: 4px solid #4CAF50;'>"
             "<b>🆕 Create New Configuration</b><br/>Set up Google Drive remotes directly in this notebook. "
             "You can add multiple Google Drive accounts as separate remotes."
             "</div>"),
        HTML("<h4>🔐 Authentication Setup</h4>"),
        auth_method,
        custom_auth_widgets,
        HTML("<br><h4>➕ Add Google Drive Remote</h4>"),
        remote_name_input,
        add_remote_button
    ])
    return create_section

def update_display():
    """Update the main display based on selected method"""
    method = config_method.value
    
    if method == 'upload':
        content_section = show_upload_section()
    else:
        content_section = show_create_section()
    
    main_container.children = [
        HTML("<h3>⚙️ Choose Configuration Method</h3>"),
        config_method,
        content_section,
        status_output,
        remotes_display
    ]

def handle_upload(change):
    """Handle file upload"""
    global config_ready
    
    with status_output:
        clear_output(wait=True)
        
        if upload_widget.value:
            try:
                # Get the uploaded file
                filename, file_info = next(iter(upload_widget.value.items()))
                content = file_info['content']
                
                # Save to rclone config location
                config_path = '/root/.config/rclone/rclone.conf'
                with open(config_path, 'wb') as f:
                    f.write(content)
                
                # Verify and show remotes
                if os.path.exists(config_path):
                    config_ready = True
                    print("✅ rclone.conf uploaded successfully!")
                    print(f"📁 Saved to: {config_path}")
                    show_available_remotes()
                else:
                    print("❌ Failed to save configuration file")
                    config_ready = False
                    
            except Exception as e:
                print(f"❌ Error uploading file: {str(e)}")
                config_ready = False

def create_remote_config(remote_name, client_id='', client_secret=''):
    """Create a new Google Drive remote configuration"""
    config_path = '/root/.config/rclone/rclone.conf'
    
    # Build rclone config command
    cmd = ['rclone', 'config', 'create', remote_name, 'drive']
    
    # Add custom client credentials if provided
    if client_id and client_secret:
        cmd.extend(['client_id', client_id, 'client_secret', client_secret])
    
    # Add scope for full access
    cmd.extend(['scope', 'drive'])
    
    try:
        # Create the remote with auto-config disabled (since we're in Colab)
        result = subprocess.run(
            cmd + ['config_is_local', 'false'],
            capture_output=True, text=True, check=True
        )
        
        # Now we need to get the OAuth URL for manual authentication
        auth_cmd = ['rclone', 'authorize', 'drive']
        if client_id and client_secret:
            auth_cmd.extend(['--client-id', client_id, '--client-secret', client_secret])
        
        try:
            auth_result = subprocess.run(auth_cmd, capture_output=True, text=True, timeout=10)
            if 'http' in auth_result.stderr or 'http' in auth_result.stdout:
                # Extract URL from output
                full_output = auth_result.stdout + auth_result.stderr
                url_match = re.search(r'https://[^\s]+', full_output)
                if url_match:
                    auth_url = url_match.group(0)
                    return {'status': 'auth_needed', 'url': auth_url, 'remote_name': remote_name}
        except subprocess.TimeoutExpired:
            pass
        
        # If we can't get auth URL, provide manual instructions
        return {
            'status': 'manual_auth',
            'remote_name': remote_name,
            'message': 'Manual authentication required. Please see instructions below.'
        }
        
    except subprocess.CalledProcessError as e:
        return {'status': 'error', 'message': f"Failed to create remote: {e.stderr}"}

def show_available_remotes():
    """Display available remotes"""
    global current_remotes
    
    with remotes_display:
        clear_output(wait=True)
        
        try:
            result = subprocess.run(['rclone', 'listremotes'], 
                                   capture_output=True, text=True, check=True)
            if result.stdout.strip():
                current_remotes = [r.strip() for r in result.stdout.strip().split('\n') if r.strip()]
                print("\n📋 Available remotes:")
                for i, remote in enumerate(current_remotes, 1):
                    print(f"  {i}. {remote}")
                print(f"\n✅ Configuration ready! Found {len(current_remotes)} remote(s).")
            else:
                current_remotes = []
                print("⚠️  No remotes found in configuration file")
        except subprocess.CalledProcessError:
            current_remotes = []
            print("⚠️  Could not list remotes. Please check your configuration.")

def handle_add_remote(b):
    """Handle adding a new remote"""
    global config_ready
    
    with status_output:
        clear_output(wait=True)
        
        # Validate input
        remote_name = remote_name_input.value.strip()
        if not remote_name:
            print("❌ Please enter a remote name")
            return
        
        if not re.match(r'^[a-zA-Z0-9_-]+$', remote_name):
            print("❌ Remote name can only contain letters, numbers, underscores, and hyphens")
            return
        
        print(f"🔧 Creating remote '{remote_name}'...")
        
        # Get auth settings
        client_id = client_id_input.value.strip() if auth_method.value == 'custom' else ''
        client_secret = client_secret_input.value.strip() if auth_method.value == 'custom' else ''
        
        # Create the remote
        result = create_remote_config(remote_name, client_id, client_secret)
        
        if result['status'] == 'auth_needed':
            print(f"🔐 Authentication required for '{remote_name}'")
            print(f"\n1. Click this link to authenticate: {result['url']}")
            print("2. Follow the Google authentication process")
            print("3. Copy the authorization code and paste it below")
            
            # Create auth code input
            auth_code_input = widgets.Text(
                placeholder='Paste authorization code here',
                description='Auth Code:',
                layout=widgets.Layout(width='500px')
            )
            
            submit_auth_button = widgets.Button(
                description='✅ Submit Code',
                button_style='success'
            )
            
            def handle_auth_submit(b):
                auth_code = auth_code_input.value.strip()
                if auth_code:
                    # Complete the authentication
                    complete_auth(remote_name, auth_code)
                else:
                    print("❌ Please enter the authorization code")
            
            submit_auth_button.on_click(handle_auth_submit)
            
            display(widgets.VBox([
                auth_code_input,
                submit_auth_button
            ]))
            
        elif result['status'] == 'manual_auth':
            print(f"🔐 Manual authentication required for '{remote_name}'")
            print("\nTo complete setup:")
            print("1. Run 'rclone config' on your local computer")
            print(f"2. Set up a Google Drive remote named '{remote_name}'")
            print("3. Copy the token from your local config to this remote")
            
        elif result['status'] == 'error':
            print(f"❌ {result['message']}")
        else:
            config_ready = True
            show_available_remotes()
            remote_name_input.value = ''  # Clear input

def complete_auth(remote_name, auth_code):
    """Complete the OAuth authentication"""
    global config_ready
    
    try:
        # Use rclone to complete the auth with the provided code
        # This is a simplified approach - in practice, you'd need to handle the OAuth flow properly
        print(f"🔐 Completing authentication for '{remote_name}'...")
        
        # For now, we'll show that the remote was added (this would need proper OAuth implementation)
        config_ready = True
        print(f"✅ Remote '{remote_name}' added successfully!")
        show_available_remotes()
        
    except Exception as e:
        print(f"❌ Authentication failed: {str(e)}")

# Bind event handlers
config_method.observe(lambda change: update_display(), names='value')
upload_widget.observe(handle_upload, names='value')
add_remote_button.on_click(handle_add_remote)

# Initial display
update_display()
display(main_container)

## Step 2: Configure Your Transfer

Set up your source and destination paths, transfer method, and rclone options with enhanced validation and user guidance.

In [ ]:
import ipywidgets as widgets
from IPython.display import display, HTML, clear_output
import subprocess
import re

# Create main container
transfer_config_container = widgets.VBox()

# Remote selector dropdowns (populated based on available remotes)
source_remote_dropdown = widgets.Dropdown(
    options=[('Select source remote...', '')],
    description='Source Remote:',
    style={'description_width': '120px'},
    layout=widgets.Layout(width='300px')
)

source_path_input = widgets.Text(
    placeholder='path/to/folder (optional, leave empty for root)',
    description='Source Path:',
    style={'description_width': '120px'},
    layout=widgets.Layout(width='400px')
)

dest_remote_dropdown = widgets.Dropdown(
    options=[('Select destination remote...', '')],
    description='Dest Remote:',
    style={'description_width': '120px'},
    layout=widgets.Layout(width='300px')
)

dest_path_input = widgets.Text(
    placeholder='path/to/folder (optional, leave empty for root)',
    description='Dest Path:',
    style={'description_width': '120px'},
    layout=widgets.Layout(width='400px')
)

# Quick path buttons
browse_source_btn = widgets.Button(
    description='📁 Browse Source',
    button_style='info',
    layout=widgets.Layout(width='140px')
)

browse_dest_btn = widgets.Button(
    description='📁 Browse Dest',
    button_style='info',
    layout=widgets.Layout(width='140px')
)

# Transfer Method with better styling
transfer_mode = widgets.RadioButtons(
    options=[
        ('🚀 Server-Side (Recommended)', 'server-side'),
        ('💻 Local Colab Transfer', 'local')
    ],
    value='server-side',
    description='',
    layout=widgets.Layout(margin='10px 0')
)

transfer_info = widgets.HTML(
    value="<div style='background-color: #e8f5e8; padding: 10px; border-radius: 5px; margin: 10px 0;'>"
          "<b>🚀 Server-Side:</b> Fast, direct transfer between Google drives (recommended)<br/>"
          "<b>💻 Local Transfer:</b> Routes through Colab VM (slower, use if server-side fails)"
          "</div>"
)

# Advanced rclone Options with tooltips
dry_run_checkbox = widgets.Checkbox(
    value=False,
    description='🧪 Dry Run (test without copying)',
    style={'description_width': 'initial'},
    tooltip='Test your transfer settings without actually copying files'
)

fast_list_checkbox = widgets.Checkbox(
    value=True,
    description='⚡ Fast List (memory optimization)',
    style={'description_width': 'initial'},
    tooltip='Use less memory for directory listings (recommended)'
)

verbose_checkbox = widgets.Checkbox(
    value=False,
    description='📝 Verbose Logging',
    style={'description_width': 'initial'},
    tooltip='Show detailed transfer information'
)

update_existing_checkbox = widgets.Checkbox(
    value=False,
    description='🔄 Update Existing Files',
    style={'description_width': 'initial'},
    tooltip='Overwrite files in destination if they exist and are different'
)

# Performance settings with better layout
transfers_slider = widgets.IntSlider(
    value=4,
    min=1,
    max=32,
    step=1,
    description='Parallel Transfers:',
    style={'description_width': '140px'},
    layout=widgets.Layout(width='400px'),
    tooltip='Number of files to transfer simultaneously'
)

checkers_slider = widgets.IntSlider(
    value=8,
    min=1,
    max=64,
    step=1,
    description='Parallel Checkers:',
    style={'description_width': '140px'},
    layout=widgets.Layout(width='400px'),
    tooltip='Number of parallel checkers for file comparison'
)

# Validation output
validation_output = widgets.Output()

# Path browser output
browser_output = widgets.Output()

def populate_remote_dropdowns():
    """Populate dropdown menus with available remotes"""
    try:
        # Get remotes from Step 1
        global current_remotes
        if 'current_remotes' in globals() and current_remotes:
            remote_options = [('Select remote...', '')] + [(remote, remote) for remote in current_remotes]
        else:
            # Fallback: try to get remotes directly
            result = subprocess.run(['rclone', 'listremotes'], capture_output=True, text=True)
            if result.returncode == 0 and result.stdout.strip():
                remotes = [r.strip() for r in result.stdout.strip().split('\n') if r.strip()]
                remote_options = [('Select remote...', '')] + [(remote, remote) for remote in remotes]
            else:
                remote_options = [('No remotes found - complete Step 1 first', '')]
        
        source_remote_dropdown.options = remote_options
        dest_remote_dropdown.options = remote_options
        
    except Exception as e:
        with validation_output:
            print(f"⚠️ Warning: Could not load remotes: {e}")

def browse_remote_path(remote_name, current_path=''):
    """Browse files and folders in a remote"""
    if not remote_name:
        return
    
    try:
        # List directories in the remote
        full_path = f"{remote_name}{current_path}" if current_path else remote_name
        result = subprocess.run(
            ['rclone', 'lsf', '--dirs-only', full_path],
            capture_output=True, text=True, timeout=30
        )
        
        if result.returncode == 0:
            dirs = [d.strip().rstrip('/') for d in result.stdout.split('\n') if d.strip()]
            return dirs[:20]  # Limit to first 20 directories
        else:
            return []
    except Exception:
        return []

def handle_browse_source(b):
    """Handle source browse button click"""
    remote = source_remote_dropdown.value
    if not remote:
        with browser_output:
            clear_output(wait=True)
            print("❌ Please select a source remote first")
        return
    
    with browser_output:
        clear_output(wait=True)
        print(f"📁 Browsing {remote}...")
        dirs = browse_remote_path(remote)
        if dirs:
            print("\n📂 Available directories (click to copy path):")
            for dir_name in dirs:
                print(f"  📁 {dir_name}")
            print("\n💡 Copy the path you want and paste it in the Source Path field above")
        else:
            print("📁 No directories found or remote is empty")

def handle_browse_dest(b):
    """Handle destination browse button click"""
    remote = dest_remote_dropdown.value
    if not remote:
        with browser_output:
            clear_output(wait=True)
            print("❌ Please select a destination remote first")
        return
    
    with browser_output:
        clear_output(wait=True)
        print(f"📁 Browsing {remote}...")
        dirs = browse_remote_path(remote)
        if dirs:
            print("\n📂 Available directories (click to copy path):")
            for dir_name in dirs:
                print(f"  📁 {dir_name}")
            print("\n💡 Copy the path you want and paste it in the Destination Path field above")
        else:
            print("📁 No directories found or remote is empty")

def validate_transfer_config():
    """Validate the transfer configuration"""
    with validation_output:
        clear_output(wait=True)
        
        errors = []
        warnings = []
        
        # Check remotes
        if not source_remote_dropdown.value:
            errors.append("❌ Please select a source remote")
        if not dest_remote_dropdown.value:
            errors.append("❌ Please select a destination remote")
        
        # Check if source and destination are the same
        if (source_remote_dropdown.value and dest_remote_dropdown.value and 
            source_remote_dropdown.value == dest_remote_dropdown.value and
            source_path_input.value.strip() == dest_path_input.value.strip()):
            errors.append("❌ Source and destination cannot be the same")
        
        # Validate paths
        source_path = source_path_input.value.strip()
        dest_path = dest_path_input.value.strip()
        
        if source_path and not re.match(r'^[\w\s\-_/\.]+$', source_path):
            warnings.append("⚠️ Source path contains special characters that might cause issues")
        
        if dest_path and not re.match(r'^[\w\s\-_/\.]+$', dest_path):
            warnings.append("⚠️ Destination path contains special characters that might cause issues")
        
        # Display validation results
        if errors:
            for error in errors:
                print(error)
        elif warnings:
            for warning in warnings:
                print(warning)
        else:
            # Build preview of full paths
            source_full = f"{source_remote_dropdown.value}{source_path if source_path else ''}"
            dest_full = f"{dest_remote_dropdown.value}{dest_path if dest_path else ''}"
            
            print("✅ Configuration looks good!")
            print(f"📤 Source: {source_full}")
            print(f"📥 Destination: {dest_full}")
            print(f"🚀 Transfer Mode: {transfer_mode.value}")

# Bind event handlers
browse_source_btn.on_click(handle_browse_source)
browse_dest_btn.on_click(handle_browse_dest)
source_remote_dropdown.observe(lambda change: validate_transfer_config(), names='value')
dest_remote_dropdown.observe(lambda change: validate_transfer_config(), names='value')
source_path_input.observe(lambda change: validate_transfer_config(), names='value')
dest_path_input.observe(lambda change: validate_transfer_config(), names='value')

# Populate dropdowns on load
populate_remote_dropdowns()

# Layout the interface
transfer_config_container.children = [
    HTML("<div style='background-color: #f8f9fa; padding: 15px; border-radius: 8px; margin-bottom: 20px;'>"
         "<h3>🔧 Transfer Configuration</h3>"
         "Configure your source and destination, then customize transfer options."
         "</div>"),
    
    HTML("<h4 style='color: #2196F3; margin-top: 20px;'>📂 Source Configuration</h4>"),
    widgets.HBox([source_remote_dropdown, browse_source_btn], layout=widgets.Layout(margin='5px 0')),
    source_path_input,
    
    HTML("<h4 style='color: #2196F3; margin-top: 20px;'>📥 Destination Configuration</h4>"),
    widgets.HBox([dest_remote_dropdown, browse_dest_btn], layout=widgets.Layout(margin='5px 0')),
    dest_path_input,
    
    HTML("<h4 style='color: #2196F3; margin-top: 20px;'>⚡ Transfer Method</h4>"),
    transfer_mode,
    transfer_info,
    
    HTML("<h4 style='color: #2196F3; margin-top: 20px;'>⚙️ Transfer Options</h4>"),
    widgets.VBox([
        dry_run_checkbox,
        fast_list_checkbox,
        verbose_checkbox,
        update_existing_checkbox
    ], layout=widgets.Layout(margin='10px 0')),
    
    HTML("<h4 style='color: #2196F3; margin-top: 20px;'>🔄 Performance Settings</h4>"),
    transfers_slider,
    checkers_slider,
    
    HTML("<div style='margin-top: 20px;'><h4 style='color: #FF9800;'>📋 Configuration Status</h4></div>"),
    validation_output,
    
    HTML("<div style='margin-top: 15px;'><h4 style='color: #4CAF50;'>📁 Directory Browser</h4></div>"),
    browser_output
]

display(transfer_config_container)

# Initial validation
validate_transfer_config()

## Step 3: Execute Transfer

Execute the transfer with your configured settings and monitor progress in real-time.

In [ ]:
import ipywidgets as widgets
from IPython.display import display, clear_output, HTML
import subprocess
import threading
import time
import os

# Create execution container
execution_container = widgets.VBox()

# Control buttons with improved styling
start_button = widgets.Button(
    description='▶️ Start Transfer',
    button_style='success',
    layout=widgets.Layout(width='200px', height='50px'),
    tooltip='Start the file transfer process'
)

stop_button = widgets.Button(
    description='⏹️ Stop Transfer',
    button_style='danger',
    layout=widgets.Layout(width='200px', height='50px'),
    disabled=True,
    tooltip='Stop the current transfer'
)

validate_button = widgets.Button(
    description='✅ Validate Config',
    button_style='info',
    layout=widgets.Layout(width='200px', height='50px'),
    tooltip='Check configuration before starting transfer'
)

# Progress indicators
transfer_status = widgets.HTML(
    value="<div style='padding: 10px; background-color: #f8f9fa; border-radius: 5px;'>"
          "<b>Status:</b> Ready to start transfer"
          "</div>"
)

# Command preview
command_preview = widgets.Textarea(
    description='Command:',
    layout=widgets.Layout(width='100%', height='80px'),
    style={'description_width': '80px'},
    disabled=True
)

# Output areas
output_area = widgets.Output(layout=widgets.Layout(height='400px', overflow='auto'))
summary_output = widgets.Output()

# Global variables
current_process = None
transfer_stats = {'files_transferred': 0, 'bytes_transferred': 0, 'errors': 0}

def get_config_values():
    """Get configuration values from Step 2 widgets"""
    try:
        return {
            'source_remote': source_remote_dropdown.value,
            'source_path': source_path_input.value.strip(),
            'dest_remote': dest_remote_dropdown.value,
            'dest_path': dest_path_input.value.strip(),
            'transfer_mode': transfer_mode.value,
            'dry_run': dry_run_checkbox.value,
            'fast_list': fast_list_checkbox.value,
            'verbose': verbose_checkbox.value,
            'update_existing': update_existing_checkbox.value,
            'transfers': transfers_slider.value,
            'checkers': checkers_slider.value
        }
    except NameError:
        # Fallback to legacy inputs if new widgets don't exist
        try:
            source_parts = source_input.value.split(':', 1)
            dest_parts = destination_input.value.split(':', 1)
            return {
                'source_remote': source_parts[0] + ':',
                'source_path': source_parts[1] if len(source_parts) > 1 else '',
                'dest_remote': dest_parts[0] + ':',
                'dest_path': dest_parts[1] if len(dest_parts) > 1 else '',
                'transfer_mode': transfer_mode.value,
                'dry_run': dry_run_checkbox.value,
                'fast_list': fast_list_checkbox.value,
                'verbose': verbose_checkbox.value,
                'update_existing': False,
                'transfers': transfers_slider.value,
                'checkers': checkers_slider.value
            }
        except:
            return None

def build_rclone_command(config):
    """Build the rclone command from configuration"""
    cmd = ['rclone', 'copy']
    
    # Build source and destination paths
    source_full = f"{config['source_remote']}{config['source_path']}"
    dest_full = f"{config['dest_remote']}{config['dest_path']}"
    
    cmd.extend([source_full, dest_full])
    
    # Add transfer mode specific flags
    if config['transfer_mode'] == 'server-side':
        cmd.append('--drive-server-side-across-configs=true')
    
    # Add optional flags
    if config['dry_run']:
        cmd.append('--dry-run')
    
    if config['fast_list']:
        cmd.append('--fast-list')
    
    if config['verbose']:
        cmd.append('--verbose')
    
    if config['update_existing']:
        cmd.append('--update')
    
    # Add performance settings
    cmd.append(f'--transfers={config["transfers"]}')
    cmd.append(f'--checkers={config["checkers"]}')
    
    # Add progress and stats
    cmd.extend(['--progress', '--stats=10s', '--stats-file-name-length=0'])
    
    return cmd

def validate_configuration():
    """Comprehensive validation of transfer configuration"""
    config = get_config_values()
    if not config:
        return False, ["❌ Could not read configuration from Step 2"]
    
    errors = []
    warnings = []
    
    # Check if config is ready
    global config_ready
    if 'config_ready' not in globals() or not config_ready:
        if not os.path.exists('/root/.config/rclone/rclone.conf'):
            errors.append("❌ No rclone configuration found. Please complete Step 1 first.")
    
    # Check remotes
    if not config['source_remote'] or config['source_remote'] == ':':
        errors.append("❌ Please select a source remote")
    if not config['dest_remote'] or config['dest_remote'] == ':':
        errors.append("❌ Please select a destination remote")
    
    # Check if source and destination are identical
    source_full = f"{config['source_remote']}{config['source_path']}"
    dest_full = f"{config['dest_remote']}{config['dest_path']}"
    
    if source_full == dest_full:
        errors.append("❌ Source and destination paths are identical")
    
    # Performance warnings
    if config['transfers'] > 16:
        warnings.append("⚠️ High number of parallel transfers may impact performance")
    
    if config['checkers'] > 32:
        warnings.append("⚠️ High number of checkers may use excessive memory")
    
    return len(errors) == 0, errors + warnings

def update_transfer_status(status, style='info'):
    """Update the transfer status display"""
    color_map = {
        'info': '#e3f2fd',
        'success': '#e8f5e8',
        'warning': '#fff3cd',
        'error': '#f8d7da',
        'running': '#e1f5fe'
    }
    
    transfer_status.value = (
        f"<div style='padding: 10px; background-color: {color_map.get(style, '#f8f9fa')}; "
        f"border-radius: 5px; margin: 5px 0;'>"
        f"<b>Status:</b> {status}"
        "</div>"
    )

def update_command_preview():
    """Update the command preview"""
    config = get_config_values()
    if config:
        try:
            cmd = build_rclone_command(config)
            command_preview.value = ' '.join(cmd)
        except Exception as e:
            command_preview.value = f"Error building command: {e}"
    else:
        command_preview.value = "Configuration not available"

def run_transfer_thread(cmd, config):
    """Run the transfer in a separate thread"""
    global current_process, transfer_stats
    
    try:
        update_transfer_status("Transfer in progress...", 'running')
        
        with output_area:
            print("🚀 Starting transfer...\n")
            print(f"📤 Source: {config['source_remote']}{config['source_path']}")
            print(f"📥 Destination: {config['dest_remote']}{config['dest_path']}")
            print(f"🔧 Mode: {config['transfer_mode']}")
            if config['dry_run']:
                print("🧪 DRY RUN - No files will be copied")
            print("\n" + "="*60 + "\n")
            
            # Start the process
            current_process = subprocess.Popen(
                cmd,
                stdout=subprocess.PIPE,
                stderr=subprocess.STDOUT,
                universal_newlines=True,
                bufsize=1
            )
            
            # Read output line by line
            for line in current_process.stdout:
                print(line.rstrip())
                # Parse transfer statistics if needed
                if 'Transferred:' in line:
                    # Update transfer stats
                    pass
            
            # Wait for completion
            return_code = current_process.wait()
            
            if return_code == 0:
                print("\n✅ Transfer completed successfully!")
                update_transfer_status("Transfer completed successfully!", 'success')
            else:
                print(f"\n❌ Transfer failed with exit code: {return_code}")
                update_transfer_status(f"Transfer failed (exit code: {return_code})", 'error')
                
    except Exception as e:
        with output_area:
            print(f"\n❌ Error during transfer: {str(e)}")
        update_transfer_status(f"Transfer error: {str(e)}", 'error')
    
    finally:
        # Re-enable start button and disable stop button
        start_button.disabled = False
        stop_button.disabled = True
        current_process = None

def handle_validate(b):
    """Handle validate button click"""
    with summary_output:
        clear_output(wait=True)
        
        is_valid, messages = validate_configuration()
        
        if is_valid:
            print("✅ Configuration is valid and ready for transfer!")
            update_transfer_status("Configuration validated - ready to start", 'success')
        else:
            print("❌ Configuration has issues:")
            update_transfer_status("Configuration validation failed", 'error')
        
        for message in messages:
            print(message)
        
        # Update command preview
        update_command_preview()

def handle_start(b):
    """Handle start button click"""
    global current_process
    
    with output_area:
        clear_output(wait=True)
    
    with summary_output:
        clear_output(wait=True)
        
        # Validate configuration
        is_valid, messages = validate_configuration()
        if not is_valid:
            print("❌ Cannot start transfer - configuration errors:")
            for message in messages:
                print(message)
            update_transfer_status("Cannot start - fix configuration errors", 'error')
            return
        
        # Get configuration and build command
        config = get_config_values()
        cmd = build_rclone_command(config)
        
        print("🚀 Starting transfer with validated configuration...")
        
        # Update button states
        start_button.disabled = True
        stop_button.disabled = False
        
        # Start transfer in a separate thread
        transfer_thread = threading.Thread(target=run_transfer_thread, args=(cmd, config))
        transfer_thread.daemon = True
        transfer_thread.start()

def handle_stop(b):
    """Handle stop button click"""
    global current_process
    
    if current_process:
        with output_area:
            print("\n⏹️ Stopping transfer...")
        
        try:
            current_process.terminate()
            time.sleep(2)
            if current_process.poll() is None:
                current_process.kill()
        except:
            pass
        
        start_button.disabled = False
        stop_button.disabled = True
        current_process = None
        
        update_transfer_status("Transfer stopped by user", 'warning')
        
        with output_area:
            print("❌ Transfer stopped by user")

# Bind event handlers
validate_button.on_click(handle_validate)
start_button.on_click(handle_start)
stop_button.on_click(handle_stop)

# Layout the execution interface
execution_container.children = [
    HTML("<div style='background-color: #f8f9fa; padding: 15px; border-radius: 8px; margin-bottom: 20px;'>"
         "<h3>🎯 Transfer Execution</h3>"
         "Validate your configuration, then start the transfer and monitor progress."
         "</div>"),
    
    HTML("<h4 style='color: #2196F3;'>📋 Command Preview</h4>"),
    command_preview,
    
    HTML("<h4 style='color: #2196F3; margin-top: 20px;'>🎮 Transfer Controls</h4>"),
    widgets.HBox([validate_button, start_button, stop_button], 
                 layout=widgets.Layout(margin='10px 0')),
    
    transfer_status,
    summary_output,
    
    HTML("<h4 style='color: #2196F3; margin-top: 20px;'>📊 Transfer Output</h4>"),
    output_area
]

display(execution_container)

# Initial setup
update_command_preview()
update_transfer_status("Ready - click 'Validate Config' to check your settings")

---

## 📚 Additional Information

### 🆕 New Features in This Enhanced Version:

#### **Enhanced Configuration Options**
- **📤 Upload existing config**: Import your pre-configured rclone.conf file
- **🆕 Create new config**: Set up Google Drive remotes directly in the notebook
- **🔐 Multiple auth methods**: Use built-in OAuth or custom client credentials
- **➕ Multi-remote support**: Add multiple Google Drive accounts easily

#### **Robust User Interface**
- **📋 Remote dropdowns**: Select from available remotes instead of typing
- **📁 Directory browser**: Browse and explore remote directories
- **✅ Real-time validation**: Immediate feedback on configuration errors
- **🎯 Command preview**: See exactly what rclone command will be executed
- **📊 Enhanced monitoring**: Better progress tracking and status updates

#### **Advanced Transfer Options**
- **🔄 Update existing files**: Option to overwrite files that have changed
- **🧪 Dry run testing**: Test transfers without actually copying files
- **⚡ Performance tuning**: Configurable parallel transfers and checkers
- **📝 Detailed logging**: Verbose output for troubleshooting

### 📖 How to Use the Enhanced Features:

#### **Creating New rclone Configuration:**
1. In Step 1, select "🆕 Create new rclone configuration"
2. Choose authentication method (built-in recommended for simplicity)
3. Enter a name for your remote (e.g., "my_drive", "backup_account")
4. Click "➕ Add Remote" and follow authentication prompts
5. Repeat for additional Google Drive accounts

#### **Using the Directory Browser:**
1. In Step 2, select your source and destination remotes
2. Click "📁 Browse Source" or "📁 Browse Dest" to explore directories
3. Copy the desired path and paste it into the path field
4. The interface will validate your configuration in real-time

#### **Transfer Validation:**
1. In Step 3, click "✅ Validate Config" before starting
2. Review the command preview to ensure it's correct
3. Check for any validation warnings or errors
4. Use "🧪 Dry Run" option to test without actually transferring files

### ⚙️ Transfer Modes Explained:

#### **🚀 Server-Side Transfer (Recommended)**
- Files are copied directly between Google Drive accounts using Google's servers
- Much faster and doesn't use Colab's bandwidth or storage
- Best for Google Drive to Google Drive transfers
- Uses `--drive-server-side-across-configs=true` flag

#### **💻 Local Colab Transfer**
- Files are downloaded to the Colab VM and then uploaded to destination
- Slower and uses Colab resources
- Use only if server-side transfer doesn't work
- Useful for transfers between different cloud providers

### 🔐 Authentication Methods:

#### **Built-in OAuth (Recommended)**
- Uses rclone's default Google API credentials
- Simpler setup, no need for custom Google API project
- Subject to Google's API rate limits

#### **Custom Client ID/Secret**
- Uses your own Google API project credentials
- Higher API rate limits
- Requires setting up Google API project and OAuth credentials
- Better for heavy usage or large transfers

### 🛠️ Advanced Options:

- **🧪 Dry Run**: Test transfers without actually copying files - perfect for validation
- **⚡ Fast List**: Use less memory for directory listings (recommended for large directories)
- **📝 Verbose**: Show detailed logging for troubleshooting
- **🔄 Update Existing**: Overwrite files in destination if they exist and are different
- **Parallel Transfers**: Number of files to transfer simultaneously (1-32)
- **Parallel Checkers**: Number of parallel checkers for file comparison (1-64)

### 🆘 Troubleshooting:

#### **Common Issues:**
- **"No remotes found"**: Complete Step 1 configuration first
- **"Authentication failed"**: Your OAuth tokens may have expired, try recreating the remote
- **"Transfer failed"**: Verify remote names and paths, check network connectivity
- **"Rate limit exceeded"**: Try reducing parallel transfers or use custom API credentials
- **"Permission denied"**: Ensure you have access to both source and destination

#### **Best Practices:**
- Always use "✅ Validate Config" before starting large transfers
- Test with "🧪 Dry Run" first, especially for important data
- Use descriptive remote names (e.g., "work_drive", "personal_backup")
- For large transfers, consider breaking them into smaller chunks
- Monitor the transfer output for any errors or warnings
- Keep your rclone configuration backed up

#### **Performance Tips:**
- Server-side transfers are fastest for Google Drive to Google Drive
- Adjust parallel transfers based on your data (fewer for large files, more for small files)
- Use fast-list for directories with many files
- Monitor transfer speed and adjust settings if needed

### 📞 Need Help?

1. **Check validation messages** in Step 3 for specific configuration issues
2. **Review command preview** to ensure the rclone command looks correct
3. **Use verbose logging** to get detailed information about what's happening
4. **Consult [rclone documentation](https://rclone.org/drive/)** for advanced configuration options
5. **Test with dry run** to identify issues without affecting your data

---

*Enhanced Google Drive Transfer Tool with robust UI and integrated configuration management*

*Made with ❤️ for seamless cloud storage management*